#### just opening the netcdf `dump=True` from `Cyclone_damage_funcs.py`
#### then saving out as mosaic geotiff 
##### currently hardcoded for each cyclone (need to change cyclone_name, nc_all, write_cog cut off string for name, and bash commands)

In [1]:
import glob, os
from os import path
import numpy as np
import xarray as xr
import rioxarray
import datacube
from datacube.testutils.io import rio_slurp_xarray
from datacube.utils.cog import write_cog

In [2]:
cyclone_name = 'Laurence'

make_dir = True

In [3]:
if make_dir == True:
    if path.exists(cyclone_name+'_tifs') == False:
        os.mkdir(cyclone_name+'_tifs')
    else:
        print('folder already exists')
else:
    pass

#### immediate impacts (cyclone year before and cyclone year after)
filenames like `Yasi_138_-191_2010-07-02_2011-07-02` which is year of cyclone and year after 

In [4]:
# e.g. 2010-07-02_2011-07-02 = immediate impacts for Yasi
# Larry _2005-07-02_2006-07-02
# Ita _2013-07-02_2014-07-02
# Marcia _2014-07-02_2015-07-02
# Lam _2014-07-02_2015-07-02
# George _2006-07-02_2007-07-02
# Ingrid_Landfall1 _2004-07-01_2005-07-02
# Ingrid_Landfall2 _2004-07-01_2005-07-02
# Ingrid_Landfall3 _2004-07-01_2005-07-02
# Moncia_Landfall1 _2005-07-02_2006-07-02
# Moncia_Landfall2 _2005-07-02_2006-07-02
# Nathan_Landfall1 _2014-07-02_2015-07-02
# Nathan_Landfall2 _2014-07-02_2015-07-02
# Laurence _2008-07-01_2009-07-02

nc_all = (glob.glob('./'+cyclone_name+'/*_2008-07-01_2009-07-02.nc'))
print(glob.glob('./'+cyclone_name+'/*_2008-07-01_2009-07-02.nc'))

['./Laurence/Laurence_-67_-153_2008-07-01_2009-07-02.nc', './Laurence/Laurence_-86_-182_2008-07-01_2009-07-02.nc', './Laurence/Laurence_-86_-191_2008-07-01_2009-07-02.nc', './Laurence/Laurence_-95_-191_2008-07-01_2009-07-02.nc', './Laurence/Laurence_-95_-182_2008-07-01_2009-07-02.nc', './Laurence/Laurence_-47_-163_2008-07-01_2009-07-02.nc', './Laurence/Laurence_-76_-163_2008-07-01_2009-07-02.nc', './Laurence/Laurence_-105_-182_2008-07-01_2009-07-02.nc', './Laurence/Laurence_-76_-172_2008-07-01_2009-07-02.nc', './Laurence/Laurence_-67_-163_2008-07-01_2009-07-02.nc', './Laurence/Laurence_-86_-172_2008-07-01_2009-07-02.nc', './Laurence/Laurence_-76_-153_2008-07-01_2009-07-02.nc', './Laurence/Laurence_-105_-191_2008-07-01_2009-07-02.nc', './Laurence/Laurence_-115_-211_2008-07-01_2009-07-02.nc', './Laurence/Laurence_-47_-172_2008-07-01_2009-07-02.nc', './Laurence/Laurence_-105_-201_2008-07-01_2009-07-02.nc', './Laurence/Laurence_-47_-153_2008-07-01_2009-07-02.nc', './Laurence/Laurence_-57_-

In [5]:
if path.exists(cyclone_name+'_tifs/cyclone_damage_immediate/') == False:
    os.mkdir(cyclone_name+'_tifs/cyclone_damage_immediate/')
else:
    print('folder already exists')

for i in nc_all:
    # open dataset
    nc_ds = xr.open_dataset(i)
    # select time=0
    singleband_da = nc_ds.damage_level.isel(time=0)
    # print name and unique values in dataset
#     print(i[7:-3]) # Yasi, Larry, Ita, Lam
#     print(i[9:-3]) # Marcia, George
    print(i[19:-3]) # Ingrid_Landfall, Monica_Landfall, Nathan_Landfall
    print(np.unique(singleband_da.values))
    # make folder
    
    # Write GeoTIFF to a location
    write_cog(geo_im=singleband_da,
              fname='./'+cyclone_name+'_tifs/cyclone_damage_immediate/'+i[19:-3]+'.tif',
              overwrite=True)

_-67_-153_2008-07-01_2009-07-02
[-1  0  1  2  3  4]
_-86_-182_2008-07-01_2009-07-02
[-1  0  1  2  3  4]
_-86_-191_2008-07-01_2009-07-02
[-1  0  1  3]
_-95_-191_2008-07-01_2009-07-02
[-1  0  1  2  3  4]
_-95_-182_2008-07-01_2009-07-02
[-1  0  1  2  3  4]
_-47_-163_2008-07-01_2009-07-02
[-1  0  1  3]
_-76_-163_2008-07-01_2009-07-02
[-1  0  1  2  3  4]
_-105_-182_2008-07-01_2009-07-02
[-1  0  1  2  3  4]
_-76_-172_2008-07-01_2009-07-02
[-1  0  1  3  4]
_-67_-163_2008-07-01_2009-07-02
[-1  0  1  2  3  4]
_-86_-172_2008-07-01_2009-07-02
[-1  0  1  2  3  4]
_-76_-153_2008-07-01_2009-07-02
[-1  0  1  3]
_-105_-191_2008-07-01_2009-07-02
[-1  0  1  3  4]
_-115_-211_2008-07-01_2009-07-02
[-1  0  1  3  4]
_-47_-172_2008-07-01_2009-07-02
[-1]
_-105_-201_2008-07-01_2009-07-02
[-1  0  1  3  4]
_-47_-153_2008-07-01_2009-07-02
[-1  0  1  3  4]
_-57_-153_2008-07-01_2009-07-02
[-1  0  1  2  3  4]
_-86_-163_2008-07-01_2009-07-02
[-1  0  1  3  4]
_-76_-182_2008-07-01_2009-07-02
[-1  0  1  3  4]


In [6]:
%%bash
cd /home/jovyan/code/TC_mangroves/Laurence_tifs/cyclone_damage_immediate

# adjust NS resolution 
for file in *.tif; 
    do echo "$file"; gdalwarp -t_srs EPSG:3577 -dstnodata -9999 -overwrite "$file" "$(basename "$file" .tif)_.tif"
done

# create vrt
gdalbuildvrt -o Laurence_immediate.vrt *_.tif

# make into tif
gdal_translate -of GTiff -co "COMPRESS=LZW" -co "TILED=YES" -co "BIGTIFF=YES" Laurence_immediate.vrt Laurence_immediate_damage.tif

# move mosaic and delete folder
mv Laurence_immediate_damage.tif /home/jovyan/code/TC_mangroves/Laurence_immediate_damage.tif
rm -rf /home/jovyan/code/TC_mangroves/Laurence_tifs/cyclone_damage_immediate/


_-105_-182_2008-07-01_2009-07-02.tif
Creating output file that is 3200P x 3200L.
Processing _-105_-182_2008-07-01_2009-07-02.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
_-105_-191_2008-07-01_2009-07-02.tif
Creating output file that is 3200P x 3200L.
Processing _-105_-191_2008-07-01_2009-07-02.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
_-105_-201_2008-07-01_2009-07-02.tif
Creating output file that is 3200P x 3200L.
Processing _-105_-201_2008-07-01_2009-07-02.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
_-115_-211_2008-07-01_2009-07-02.tif
Creating output file that is 3200P x 3200L.
Processing _-115_-211_2008-07-01_2009-07-02.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
_-47_-153_2008-07-01_2009-07-02.tif
Creating output file that is 798P x 3200L.
Processing _-47_-153_2008-07-01_2009-07-02.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
_-47_-163_200

#### long term impacts
filenames with _all.nc

In [7]:
# all = long term impacts
nc_all = (glob.glob('./'+cyclone_name+'/*_all.nc'))
print(glob.glob('./'+cyclone_name+'/*_all.nc'))

['./Laurence/Laurence_-86_-163_all.nc', './Laurence/Laurence_-95_-191_all.nc', './Laurence/Laurence_-57_-153_all.nc', './Laurence/Laurence_-115_-211_all.nc', './Laurence/Laurence_-76_-172_all.nc', './Laurence/Laurence_-67_-163_all.nc', './Laurence/Laurence_-86_-182_all.nc', './Laurence/Laurence_-105_-182_all.nc', './Laurence/Laurence_-76_-153_all.nc', './Laurence/Laurence_-67_-153_all.nc', './Laurence/Laurence_-76_-163_all.nc', './Laurence/Laurence_-95_-182_all.nc', './Laurence/Laurence_-47_-153_all.nc', './Laurence/Laurence_-47_-163_all.nc', './Laurence/Laurence_-105_-191_all.nc', './Laurence/Laurence_-86_-191_all.nc', './Laurence/Laurence_-86_-172_all.nc', './Laurence/Laurence_-47_-172_all.nc', './Laurence/Laurence_-105_-201_all.nc', './Laurence/Laurence_-76_-182_all.nc']


In [8]:
if path.exists(cyclone_name+'_tifs/cyclone_damage_longterm/') == False:
    os.mkdir(cyclone_name+'_tifs/cyclone_damage_longterm/')
else:
    print('folder already exists')

for i in nc_all:
    # open dataset
    nc_ds = xr.open_dataset(i)
    # select time=0
    singleband_da = nc_ds.damage_level.isel(time=0)
    # print name and unique values in dataset
#     print(i[7:-3])
#     print(i[9:-3]) # Marcia, George
    print(i[19:-3]) # Ingrid_Landfall, Monica_Landfall, Nathan_Landfall
    print(np.unique(singleband_da.values))
    # make folder
    
    # Write GeoTIFF to a location
    write_cog(geo_im=singleband_da,
              fname='./'+cyclone_name+'_tifs/cyclone_damage_longterm/'+i[19:-3]+'.tif',
              overwrite=True)

_-86_-163_all
[-1  0  1  3  4]
_-95_-191_all
[-1  0  1  2  3  4  5]
_-57_-153_all
[-1  0  1  2  3  4]
_-115_-211_all
[-1  0  1  3  4  5]
_-76_-172_all
[-1  0  1  2  3  4  5]
_-67_-163_all
[-1  0  1  2  3  4  5]
_-86_-182_all
[-1  0  1  2  3  4  5]
_-105_-182_all
[-1  0  1  2  3  4]
_-76_-153_all
[-1  0  1  3]
_-67_-153_all
[-1  0  1  2  3  4]
_-76_-163_all
[-1  0  1  2  3  4  5]
_-95_-182_all
[-1  0  1  2  3  4  5]
_-47_-153_all
[-1  0  1  2  3  4]
_-47_-163_all
[-1  0  1  3  4]
_-105_-191_all
[-1  0  1  2  3  4  5]
_-86_-191_all
[-1  0  1  3  4]
_-86_-172_all
[-1  0  1  2  3  4]
_-47_-172_all
[-1]
_-105_-201_all
[-1  0  1  2  3  4]
_-76_-182_all
[-1  0  1  2  3]


In [9]:
%%bash
cd /home/jovyan/code/TC_mangroves/Laurence_tifs/cyclone_damage_longterm

# adjust NS resolution 
for file in *.tif; 
    do echo "$file"; gdalwarp -t_srs EPSG:3577 -dstnodata -9999 -overwrite "$file" "$(basename "$file" .tif)_.tif"
done

# create vrt
gdalbuildvrt -o Laurence_all.vrt *_.tif

# make into tif
gdal_translate -of GTiff -co "COMPRESS=LZW" -co "TILED=YES" -co "BIGTIFF=YES" Laurence_all.vrt Laurence_longterm_damage.tif


# move mosaic and delete folder
mv Laurence_longterm_damage.tif /home/jovyan/code/TC_mangroves/Laurence_longterm_damage.tif
rm -rf /home/jovyan/code/TC_mangroves/Laurence_tifs/


_-105_-182_all.tif
Creating output file that is 3200P x 3200L.
Processing _-105_-182_all.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
_-105_-191_all.tif
Creating output file that is 3200P x 3200L.
Processing _-105_-191_all.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
_-105_-201_all.tif
Creating output file that is 3200P x 3200L.
Processing _-105_-201_all.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
_-115_-211_all.tif
Creating output file that is 3200P x 3200L.
Processing _-115_-211_all.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
_-47_-153_all.tif
Creating output file that is 798P x 3200L.
Processing _-47_-153_all.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
_-47_-163_all.tif
Creating output file that is 798P x 3200L.
Processing _-47_-163_all.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
_-47_-172_all.tif
Creating output 